In [1]:
import pandas as pd

df = pd.read_csv('users_behavior.csv')

In [2]:
df.head

<bound method NDFrame.head of       calls  minutes  messages   mb_used  is_ultra
0      40.0   311.90      83.0  19915.42         0
1      85.0   516.75      56.0  22696.96         0
2      77.0   467.66      86.0  21060.45         0
3     106.0   745.53      81.0   8437.39         1
4      66.0   418.74       1.0  14502.75         0
...     ...      ...       ...       ...       ...
3209  122.0   910.98      20.0  35124.90         1
3210   25.0   190.36       0.0   3275.61         0
3211   97.0   634.44      70.0  13974.06         0
3212   64.0   462.32      90.0  31239.78         0
3213   80.0   566.09       6.0  29480.52         1

[3214 rows x 5 columns]>

In [3]:
from sklearn.model_selection import train_test_split

# Dividir os dados em conjunto de treinamento e conjunto de teste
train_data, test_data = train_test_split(df, test_size=0.25, random_state=12345)

# Dividir o conjunto de treinamento em conjunto de treinamento e conjunto de validação
train_data, val_data = train_test_split(train_data, test_size=0.25, random_state=42)  # 0.25 * 0.8 = 0.2 do total

# Visualizar as dimensões dos conjuntos
print(f'Treinamento: {train_data.shape}')
print(f'Validação: {val_data.shape}')
print(f'Teste: {test_data.shape}')

Treinamento: (1807, 5)
Validação: (603, 5)
Teste: (804, 5)


In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Definindo os hiperparâmetros a serem testados
n_estimators_options = [100, 200]
max_depth_options = [None, 10, 20, 30]

# Inicializando variáveis para armazenar os melhores resultados
best_accuracy = 0
best_params = {}
best_rf = None

# Loop sobre os hiperparâmetros
for n_estimators in n_estimators_options:
    for max_depth in max_depth_options:
        # Inicializando o modelo com os hiperparâmetros atuais e random_state para reprodutibilidade
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        
        # Treinando o modelo
        model.fit(train_data.drop('is_ultra', axis=1), train_data['is_ultra'])
        
        # Fazendo previsões no conjunto de validação
        predictions = model.predict(val_data.drop('is_ultra', axis=1))
        
        # Calculando a acurácia
        accuracy = accuracy_score(val_data['is_ultra'], predictions)
        
        # Verificando se a acurácia atual é a melhor
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = {'n_estimators': n_estimators, 'max_depth': max_depth}
            best_rf = model

# Resultados
print("Melhores hiperparâmetros:", best_params)
print("Melhor acurácia:", best_accuracy)


Melhores hiperparâmetros: {'n_estimators': 200, 'max_depth': 10}
Melhor acurácia: 0.8043117744610282


In [5]:
   print(train_data.columns)

Index(['calls', 'minutes', 'messages', 'mb_used', 'is_ultra'], dtype='object')


O melhor resultado foi alcançado com 100 árvores. A profundidade máxima de 10 sugere que o modelo conseguiu aprender padrões significativos sem se tornar excessivamente complexo. Cerca de 80% do modelo consegue classificar corretamente o conjunto de validação. 

In [9]:

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Normalizando os dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(train_data.drop('is_ultra', axis=1))
X_val_scaled = scaler.transform(val_data.drop('is_ultra', axis=1))

# Definindo os hiperparâmetros a serem testados
C_options = [0.01, 0.1, 1, 10, 100]
solver_options = ['liblinear', 'saga']

best_accuracy_logistic = 0
best_params_logistic = {}

for C in C_options:
    for solver in solver_options:
        model = LogisticRegression(C=C, solver=solver, max_iter=500, random_state=42)
        model.fit(X_train_scaled, train_data['is_ultra'])
        predictions = model.predict(X_val_scaled)
        accuracy = accuracy_score(val_data['is_ultra'], predictions)
        if accuracy > best_accuracy_logistic:
            best_accuracy_logistic = accuracy
            best_params_logistic = {'C': C, 'solver': solver}

print("Melhores hiperparâmetros da Regressão Logística:", best_params_logistic)
print("Melhor acurácia da Regressão Logística:", best_accuracy_logistic)


Melhores hiperparâmetros da Regressão Logística: {'C': 0.01, 'solver': 'liblinear'}
Melhor acurácia da Regressão Logística: 0.7479270315091211


Conclusão: RandomForest aprensentou a acurácia significativamente melhor (80,43%) em comparação com o modelo de Regressão Logística (74,79%). Assim, se mostra mais eficaz em classificar corretamente os planos (Ultra ou Smart) com base nas características dos usuários.

In [10]:
from sklearn.metrics import accuracy_score, precision_score

# Fazendo previsões no conjunto de teste usando o melhor modelo Random Forest
test_predictions = best_rf.predict(test_data.drop('is_ultra', axis=1))

# Calculando as métricas de desempenho
accuracy = accuracy_score(test_data['is_ultra'], test_predictions)
precision = precision_score(test_data['is_ultra'], test_predictions)

# Mostrando os resultados
print("Resultados no Conjunto de Teste:")
print(f"Acurácia: {accuracy:.4f}")
print(f"Precisão: {precision:.4f}")


Resultados no Conjunto de Teste:
Acurácia: 0.8072
Precisão: 0.7722


In [11]:
# Normalizar o conjunto de teste (isso precisa estar antes do predict)
X_test_scaled = scaler.transform(test_data.drop('is_ultra', axis=1))

# Treinar o melhor modelo com os melhores hiperparâmetros
best_model_logistic = LogisticRegression(C=best_params_logistic['C'], solver=best_params_logistic['solver'], max_iter=500, random_state=42)
best_model_logistic.fit(X_train_scaled, train_data['is_ultra'])

# Fazer previsões no conjunto de teste
test_predictions = best_model_logistic.predict(X_test_scaled)

# Calcular métricas
accuracy = accuracy_score(test_data['is_ultra'], test_predictions)
precision = precision_score(test_data['is_ultra'], test_predictions)

print("Resultados no Conjunto de Teste:")
print(f"Acurácia: {accuracy:.4f}")
print(f"Precisão: {precision:.4f}")


Resultados no Conjunto de Teste:
Acurácia: 0.7600
Precisão: 0.8429


Hipótese: Apesar da Regressão Logística ter tido uma acurácia menor que da RandomForest nos hiperparâmetros, após o conjunto de teste pode se perceber que a acurácia e precisão da random forest foi significamente menor, mostrando que ela provavelemnte generalizou os dados, não sendo confiável. 